In [38]:
import torch
import torchvision
import torchvision.transforms as transforms

# GPU 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

transfrom = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 32 # 배치 크기

# 데이터셋 다운로드
full_trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transfrom)
import torch
import torchvision
import torchvision.transforms as transforms

# 데이터 증강
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

# 검증/테스트 데이터
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# 데이터셋 다운로드
full_trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [39]:
from torch.utils.data import random_split

# Train/Validation 분할 (80:20)
train_size = int(0.8 * len(full_trainset))
val_size = len(full_trainset) - train_size

datasets = random_split(full_trainset, [train_size, val_size])
trainset = datasets[0]  # 학습용
valset = datasets[1]     # 검증용


# 데이터셋 불러오기
# DataLoader : 데이터셋을 배치 단위로 관리하는 역할
    # batch_size : 배치 크기
    # shuffle : 데이터를 섞을지 여부
    # num_workers : 데이터 로드에 사용할 쓰레드 수
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=0)
val_loader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False, num_workers=0)
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=0)

In [40]:
import torch
import torch.nn as nn

# SEBlock : Squeeze-and-Excitation Block
class SEBlock(nn.Module):
    def __init__(self, in_channels, reduction=16):
        super(SEBlock, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1) # Squeeze

        # Excitation
        self.fc = nn.Sequential(
            nn.Linear(in_channels, in_channels // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(in_channels // reduction, in_channels, bias=False),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        b, c, _, _ = x.size()
        z = self.avg_pool(x).view(b, c) # 딥러닝에서는 이미지를 batch 수 만큼 한번에 진행
        s = self.fc(z).view(b, c, 1, 1) # 채널 수 만큼 출력
        return x * s # scale

In [41]:
import torch.nn.functional as F

class ResNetBottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1, use_se=False, reduction=16):
        super(ResNetBottleneck, self).__init__()

        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)

        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.conv3 = nn.Conv2d(planes, planes * self.expansion, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * self.expansion)

        # SEBlock 추가 여부
        if use_se:
            self.se = SEBlock(planes * self.expansion, reduction)
        else:
            self.se = nn.Identity() # SEBlock 추가 안할 경우 그냥 통과

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes * self.expansion:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes * self.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * self.expansion)
            )
            
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out = self.se(out)
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [42]:
class Net(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10, use_se=False, reduction=16):
        super(Net, self).__init__()
        self.in_planes = 64

        # 채널 수 3 -> 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)

        # _make_layer 함수 호출하여 여러 개의 블록 그룹 생성
        self.layer1 = self._make_layer(block, 64, num_blocks[0], 1, use_se, reduction)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], 2, use_se, reduction)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], 2, use_se, reduction)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], 2, False, reduction) # 마지막 블록은 SEBlock 추가 안함

        # average pooling 레이어 사용
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        # 완전 연결 레이어(Fully Connected Layer) 512 * block.expansion -> 10개의 클래스로 분류
        self.fc = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride, use_se=False, reduction=16):
        strides = [stride] + [1] * (num_blocks - 1) # 첫 번째 블록은 stride, 나머지는 1
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride, use_se, reduction))
            self.in_planes = planes * block.expansion # 채널 수 증가
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avgpool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [43]:
# 50 layer resnet
resnet_50 = Net(ResNetBottleneck, [3, 4, 6, 3])
resnet_50_loss = []
resnet_50_error = []

se_resnet_50 = Net(ResNetBottleneck, [3, 4, 6, 3], use_se=True, reduction=16)
se_resnet_50_loss = []
se_resnet_50_error = []

# 모델을 GPU로 이동
resnet_50 = resnet_50.to(device)
se_resnet_50 = se_resnet_50.to(device)

In [44]:
import torch.optim as optim
import numpy as np

# 크로스 엔트로피 손실 함수 사용
criterion = nn.CrossEntropyLoss()

# 초기 학습률을 더 낮게 설정 (경사 폭발 방지)
# ResNet-50과 같은 깊은 네트워크에서는 더 보수적인 접근 필요

# 모멘텀 0.9, weight_decay 1e-4
resnet_50_optimizer = optim.SGD(resnet_50.parameters(), lr=0.001, momentum=0.9, weight_decay=1e-4)
se_resnet_50_optimizer = optim.SGD(se_resnet_50.parameters(), lr=0.001, momentum=0.9, weight_decay=1e-4)

# 학습률 스케줄링
resnet_50_scheduler = optim.lr_scheduler.ReduceLROnPlateau(resnet_50_optimizer, mode='min', factor=0.1, patience=5)
se_resnet_50_scheduler = optim.lr_scheduler.ReduceLROnPlateau(se_resnet_50_optimizer, mode='min', factor=0.1, patience=5)

# 20번 에포크 학습
num_epochs = 20

In [45]:
# 평가 함수 (top1 error rate, top5 error rate)
def evaluate_model(model, data_loader, is_top5=False):
    model.eval()
    total_correct = 0
    total_samples = 0

    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs = inputs.to(device)  # GPU로 이동
            labels = labels.to(device)  # GPU로 이동
            
            outputs = model(inputs)
            
            if is_top5:
                _, top5 = outputs.topk(5, dim=1)
                total_correct += (top5 == labels.unsqueeze(1)).any(dim=1).sum().item()
            else:
                _, predicted = torch.max(outputs.data, 1)
                total_correct += (predicted == labels).sum().item()
            total_samples += labels.size(0)

    return (1 - total_correct / total_samples) * 100           

In [46]:
# SE-ResNet-50 model 학습 (스케줄링 적용)
for epoch in range(num_epochs):
    # 모델을 학습 모드로 설정
    se_resnet_50.train()
    
    running_loss_se_resnet_50 = 0.0
    
    print(f"\nEpoch {epoch + 1}/{num_epochs}")
    print(f"Current LR - SE-ResNet50: {se_resnet_50_optimizer.param_groups[0]['lr']:.6f}")
    
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs = inputs.to(device)  # GPU로 이동
        labels = labels.to(device)  # GPU로 이동

        # SE-ResNet-50 학습
        se_resnet_50_optimizer.zero_grad()
        outputs_se_resnet_50 = se_resnet_50(inputs)
        loss_se_resnet_50 = criterion(outputs_se_resnet_50, labels)
        loss_se_resnet_50.backward()
        se_resnet_50_optimizer.step()
        

        # 배치 손실 합계 계산
        running_loss_se_resnet_50 += loss_se_resnet_50.item()

        # 100 배치마다 검증 및 출력
        if (i + 1) % 100 == 0:
            val_error_se_resnet_50 = evaluate_model(se_resnet_50, val_loader)

            se_resnet_50_error.append(val_error_se_resnet_50)

            avg_loss_se_resnet_50 = running_loss_se_resnet_50 / 100
            
            se_resnet_50_loss.append(avg_loss_se_resnet_50)

            print(f"Batch {i + 1}: SE-ResNet50 Loss = {avg_loss_se_resnet_50:.4f}, Error = {val_error_se_resnet_50:.2f}%")
            
            # running_loss 초기화
            running_loss_se_resnet_50 = 0.0
    
    # 에포크 끝에서 스케줄러 업데이트 (검증 오류율 기준)
    if len(se_resnet_50_error) > 0:
        se_resnet_50_scheduler.step(se_resnet_50_error[-1])

print("학습 완료!")


Epoch 1/20
Current LR - SE-ResNet50: 0.001000
Batch 100: SE-ResNet50 Loss = 2.2631, Error = 75.99%
Batch 200: SE-ResNet50 Loss = 2.1891, Error = 73.93%
Batch 300: SE-ResNet50 Loss = 1.9621, Error = 70.32%
Batch 400: SE-ResNet50 Loss = 1.8811, Error = 70.68%
Batch 500: SE-ResNet50 Loss = 1.8298, Error = 71.09%
Batch 600: SE-ResNet50 Loss = 1.7387, Error = 63.33%
Batch 700: SE-ResNet50 Loss = 1.6845, Error = 69.06%
Batch 800: SE-ResNet50 Loss = 1.7064, Error = 64.08%
Batch 900: SE-ResNet50 Loss = 1.6457, Error = 60.02%
Batch 1000: SE-ResNet50 Loss = 1.6090, Error = 61.33%
Batch 1100: SE-ResNet50 Loss = 1.5986, Error = 59.42%
Batch 1200: SE-ResNet50 Loss = 1.5659, Error = 58.15%

Epoch 2/20
Current LR - SE-ResNet50: 0.001000
Batch 100: SE-ResNet50 Loss = 1.6845, Error = 61.08%
Batch 200: SE-ResNet50 Loss = 1.8212, Error = 60.47%
Batch 300: SE-ResNet50 Loss = 1.6972, Error = 60.92%
Batch 400: SE-ResNet50 Loss = 1.5822, Error = 55.54%
Batch 500: SE-ResNet50 Loss = 1.5125, Error = 53.38%
Ba

In [ ]:
# ResNet-50 model 학습 (스케줄링 적용)
for epoch in range(num_epochs):
    # 모델을 학습 모드로 설정
    resnet_50.train()
    
    running_loss_resnet_50 = 0.0
    
    print(f"\nEpoch {epoch + 1}/{num_epochs}")
    print(f"Current LR - ResNet50: {resnet_50_optimizer.param_groups[0]['lr']:.6f}")
    
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs = inputs.to(device)  # GPU로 이동
        labels = labels.to(device)  # GPU로 이동

        # ResNet-50 학습
        resnet_50_optimizer.zero_grad()
        outputs_resnet_50 = resnet_50(inputs)
        loss_resnet_50 = criterion(outputs_resnet_50, labels)
        loss_resnet_50.backward()
        resnet_50_optimizer.step()
        

        # 배치 손실 합계 계산
        running_loss_resnet_50 += loss_resnet_50.item()

        # 100 배치마다 검증 및 출력
        if (i + 1) % 100 == 0:
            val_error_resnet_50 = evaluate_model(resnet_50, val_loader)

            resnet_50_error.append(val_error_resnet_50)

            avg_loss_resnet_50 = running_loss_resnet_50 / 100
            
            resnet_50_loss.append(avg_loss_resnet_50)

            print(f"Batch {i + 1}: ResNet50 Loss = {avg_loss_resnet_50:.4f}, Error = {val_error_resnet_50:.2f}%")
            
            # running_loss 초기화
            running_loss_resnet_50 = 0.0
    
    # 에포크 끝에서 스케줄러 업데이트 (검증 오류율 기준)
    if len(resnet_50_error) > 0:
        resnet_50_scheduler.step(resnet_50_error[-1])

print("학습 완료!")


Epoch 1/100
Current LR - ResNet50: 0.001000
Batch 100: ResNet50 Loss = 2.3851, Error = 75.33%
Batch 200: ResNet50 Loss = 2.1475, Error = 79.39%
Batch 300: ResNet50 Loss = 2.0202, Error = 71.74%
Batch 400: ResNet50 Loss = 1.9066, Error = 71.41%
Batch 500: ResNet50 Loss = 1.8452, Error = 68.24%
Batch 600: ResNet50 Loss = 1.7856, Error = 64.99%
Batch 700: ResNet50 Loss = 1.7570, Error = 65.96%
Batch 800: ResNet50 Loss = 1.7672, Error = 63.48%
Batch 900: ResNet50 Loss = 1.6554, Error = 61.32%
Batch 1000: ResNet50 Loss = 1.6462, Error = 62.57%
Batch 1100: ResNet50 Loss = 1.6521, Error = 61.57%
Batch 1200: ResNet50 Loss = 1.6032, Error = 61.94%

Epoch 2/100
Current LR - ResNet50: 0.001000
Batch 100: ResNet50 Loss = 1.7759, Error = 61.44%
Batch 200: ResNet50 Loss = 1.9091, Error = 62.93%
Batch 300: ResNet50 Loss = 1.6761, Error = 64.79%
Batch 400: ResNet50 Loss = 1.6089, Error = 58.88%
Batch 500: ResNet50 Loss = 1.5590, Error = 60.20%
Batch 600: ResNet50 Loss = 1.5876, Error = 58.11%
Batch 7

KeyboardInterrupt: 